In [33]:
from tensorflow.keras.layers import Dense,Flatten

In [34]:
import os

In [35]:
data='new_brain_tumor_project/training/training'
classs=['glioma','meningioma','notumor','pituitary']

In [36]:
import tensorflow as tf

In [37]:
# removed=0
# for folder_name in classs:

#     folder_path=os.path.join('training/training',folder_name)
#     # print(folder_name)
#     for f_name in os.listdir(folder_path):
#         # print(f_name)
#         f_path=os.path.join(folder_path,f_name)
#         # print(f_path)

#         try:
#             img_byte=tf.io.read_file(f_path)
#             tf.io.decode_image(img_byte)
#             # print(f_path)
#         except Exception as e:
#             print(f' bad image found and deleted on {f_path}')
#             removed +=1
#             os.remove(f_path)


# print(f' bad images deleted {removed}')
    

In [38]:
directory='training/training/'
dir='testing/testing'

In [43]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    directory,
    label_mode='categorical',
    batch_size=15,
    subset='training',
    validation_split=0.2,
    seed=124,
    image_size=(64,64)


)

var_ds=tf.keras.utils.image_dataset_from_directory(
    dir,
    label_mode='categorical',
    batch_size=15,
    subset='validation',
    validation_split=0.2,
    seed=124,
    image_size=(64,64)


)

Found 20000 files belonging to 4 classes.
Using 16000 files for training.
Found 800 files belonging to 4 classes.
Using 160 files for validation.


In [44]:
auto_tune =tf.data.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=auto_tune)
var_ds =var_ds.cache().prefetch(buffer_size=auto_tune)

In [45]:
img_shape=(64,64,3)
base_model =tf.keras.applications.VGG16(
    input_shape=img_shape,
    include_top =False,
    weights ='imagenet'
)
base_model.trainable=False

In [46]:
best_model=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(64,64,3)),
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    # tf.keras.layers.RandomZoom(0.2),
    # tf.keras.layers.RandomBrightness(0.3),
    # tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(200,activation='relu'),
                        #   ,kernel_regularizer=tf.keras.regularizers.L2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(100,activation='relu'),
                        #   kernel_regularizer=tf.keras.regularizers.L2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4,activation='softmax')
    

])
 
best_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
best_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_3 (RandomFlip)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_3               │ (None, 64, 64, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 2, 2, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 200)            │       102,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,837,792 (56.60 MB)

 Trainable params: 123,104 (480.88 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [47]:
best_model.fit(
    train_ds,
    epochs=10,
    validation_data=var_ds,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True,mode='max'),
        tf.keras.callbacks.ModelCheckpoint('braintumor_prediction.keras',monitor='val_loss',save_best_only=True,mode='min',verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='va_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            mode='min'
        )
        
    ]
        
    
)

Epoch 1/10
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.6548 - loss: 0.8231
Epoch 1: val_loss improved from inf to 0.54699, saving model to braintumor_prediction.keras
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 277s 258ms/step - accuracy: 0.6549 - loss: 0.8229 - val_accuracy: 0.7437 - val_loss: 0.5470 - learning_rate: 0.0010
Epoch 2/10


c:\Users\Kelvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `va_loss` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


1067/1067 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.7909 - loss: 0.5230
Epoch 2: val_loss improved from 0.54699 to 0.51684, saving model to braintumor_prediction.keras
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 246s 231ms/step - accuracy: 0.7909 - loss: 0.5230 - val_accuracy: 0.7875 - val_loss: 0.5168 - learning_rate: 0.0010
Epoch 3/10
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - accuracy: 0.8149 - loss: 0.4597
Epoch 3: val_loss did not improve from 0.51684
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 248s 232ms/step - accuracy: 0.8149 - loss: 0.4597 - val_accuracy: 0.7688 - val_loss: 0.5600 - learning_rate: 0.0010
Epoch 4/10
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.8270 - loss: 0.4290
Epoch 4: val_loss improved from 0.51684 to 0.51520, saving model to braintumor_prediction.keras
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 250s 234ms/step - accuracy: 0.8270 - loss: 0.4290 - val_accuracy: 0.7688 - val_loss: 0.5152 - learning_rate: 0.0010
Epoch 5/10
1067/1067 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - acc

In [48]:
import numpy as np

In [49]:
import tensorflow as tf

In [51]:
from tensorflow.keras.models import load_model

In [54]:
model=tf.keras.models.load_model('braintumor_prediction.keras')
glioma='Healthy_brain.jpg'
img=tf.keras.utils.load_img(glioma,target_size=(64,64))
array_img=tf.keras.utils.img_to_array(img)
norm=array_img/255
batch_img=np.expand_dims(norm,axis=0)

In [55]:
prediction=model.predict(batch_img)
kelvin=np.argmax(prediction)
print(classs[kelvin])
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
glioma
[[9.287150e-01 8.887657e-03 6.157156e-02 8.258220e-04]]
